# 4. domača naloga

Pri tej nalogi boste napisali svoj simulator Turingovih strojev. Zaradi preprostosti bomo za abecedo vzeli kar znake tipa `char`, za prazni znak bomo izbrali presledek `' '`, stanja pa bomo predstavili z nizi. Za možne premike zafiksiramo tip `direction`:

In [27]:
type direction = Left | Right
type state = string

type direction = Left | Right


type state = string


## Implementacija trakov

Napišite modul `Tape`, ki implementira spodnjo signaturo, kjer je:

- `t` tip v obe smeri neomejenih trakov in glavo na danem mestu;
- `make`, ki naredi nov trak z znaki iz niza ter glavo na prvem znaku;
- `print`, ki izpiše vsebino traku (brez presledkov na začetku in koncu) ter pod njim z `^` označi mesto glave;
- `read`, ki vrne znak pod glavo;
- `write`, ki pod glavo zapiše dani znak;
- `move`, ki glavo premakne v dano smer.

Zadnji dve funkciji naj vrneta nov trak, obstoječega pa naj pustita nespremenjenega.

Ker je tip `t` abstrakten, si lahko privoščite poljubno implementacijo, zato poskrbite tako za učinkovitost kot za preglednost kode.

In [28]:
module type TAPE = sig
  type t

  val make : string -> t
  val print : t -> unit
  val read : t -> char
  val move : direction -> t -> t
  val write : char -> t -> t
end

(* pomozne funkcije *)
let string_reverse str = (*obrne niz*)
  String.of_seq (List.to_seq (List.rev (List.of_seq (String.to_seq str))))
let string_of_list l =   (*pretvori seznam v niz*)
  String.of_seq (List.to_seq l)
let podcrtaj str = 
  let len = String.length str in
  let underline = String.make len '^' in
  str ^ "\n" ^ underline
let stresica s =
  (String.make (String.length s) ' ') ^ "^"
let hd l =
  match l with 
  | [] -> ' '
  | x :: _ -> x
(*----------------------*)
  
module Tape : TAPE = struct
  type t = {
    levi : char list;
    glava : char;
    desni : char list
  }
  let make str =
    {glava = String.get str 0; levi = [];
    desni = List.of_seq (String.to_seq (String.sub str 1 (String.length str - 1)))}
  let read t = t.glava
  let write ch t = {t with glava = ch}
  let print t =
    let leva_stran = (string_of_list (List.rev t.levi)) and
    glava_traku = String.make 1 t.glava and
    desna_stran = string_of_list t.desni in
    let cel_niz = (leva_stran ^ glava_traku ^ desna_stran ^ "\n" ^ stresica leva_stran) in
    print_string (cel_niz);
    print_newline ()
    let move dir t =
      match (dir, t.levi, t.desni) with
      | (Left, x :: xs, _) ->
          { levi = xs; glava = x; desni = t.glava :: t.desni }
      | (Left, [], _) ->
          { levi = []; glava = ' '; desni = t.glava :: t.desni }
      | (Right, _, y :: ys) ->
          { levi = t.glava :: t.levi; glava = y; desni = ys }
      | (Right, _, []) ->
          { levi = t.glava :: t.levi; glava = ' '; desni = [] }
    
end

module type TAPE =
  sig
    type t
    val make : string -> t
    val print : t -> unit
    val read : t -> char
    val move : direction -> t -> t
    val write : char -> t -> t
  end


val string_reverse : String.t -> String.t = <fun>


val string_of_list : char list -> String.t = <fun>


val podcrtaj : string -> string = <fun>


val stresica : string -> string = <fun>


val hd : char list -> char = <fun>


module Tape : TAPE


In [29]:
let primer_trak = Tape.(
  make "ABCDE"
  |> move Left
  |> move Left
  |> move Right
  |> move Right
  |> move Right
  |> move Right
  |> write '!'
  |> print
)

  AB!DE
    ^


val primer_trak : unit = ()


## Implementacija Turingovih strojev

Napišite modul `Machine`, ki implementira spodnjo signaturo, kjer je:

- `t` tip Turingovih strojev;
- `make`, ki naredi nov stroj z danim začetnim stanjem in seznamom preostalih stanj ter prazno prehodno funkcijo;
- `initial`, ki vrne začetno stanje stroja;
- `add_transition`, ki prehodno funkcijo razširi s prehodom $(q, a) \mapsto (q', a', d)$;
- `step`, ki za dano stanje in trak izvede en korak stroja, če je to mogoče.

Zadnji dve funkciji naj vrneta spremenjene vrednosti, obstoječe argumente pa naj pustita nespremenjene. Prav tako pri zadnjih dveh funkcijah lahko predpostavite, da ju bomo klicali le na poprej podanih stanjih.

Tudi tu je tip `t` abstrakten, zato poskrbite za učinkovitost in preglednost kode.

Vpeljemo modul AVL za delo z AVL drevesi s predavanj.

In [30]:
module type AVL = sig
  type 'a t

  val prazna : 'a t
  val vsebuje : 'a t -> 'a -> bool
  val velikost : 'a t -> int
  val podvoji : 'a t -> 'a t
  val zavrti_levo : 'a t -> 'a t
  val zavrti_desno : 'a t -> 'a t
  val visina : 'a t -> int
  val razlika : 'a t -> int
  val uravnotezi : 'a t -> 'a t
  val isci : 'a -> 'a t -> bool
  val dodaj : 'a -> 'a t -> 'a t
end


module Avl : AVL = struct
  type 'a t = Prazno | Sestavljeno of 'a t * 'a * 'a t

  let rec vsebuje mn x =
    match mn with
    | Prazno -> false
    | Sestavljeno (l, y, d) when x = y -> true
    | Sestavljeno (l, y, d) when x < y -> vsebuje l x
    | Sestavljeno (l, y, d) when x > y -> vsebuje d x
    | _ -> assert false

  let prazna = Prazno

  let rec velikost = function
    | Prazno -> 0
    | Sestavljeno (l, _, d) -> 1 + velikost l + velikost d

  let rec podvoji = function
    | Prazno -> Prazno
    | Sestavljeno (l, x, d) -> Sestavljeno (podvoji l, x, podvoji d)

  let zavrti_levo = function
    | Sestavljeno (l, x, Sestavljeno (dl, y, dd)) ->
        Sestavljeno (Sestavljeno (l, x, dl), y, dd)
    | _ -> failwith "Tega drevesa ne morem zavrteti"

  let zavrti_desno = function
    | Sestavljeno (Sestavljeno (ll, y, ld), x, d) ->
        Sestavljeno (ll, y, Sestavljeno (ld, x, d))
    | _ -> failwith "Tega drevesa ne morem zavrteti"

  let rec visina drevo =
    match drevo with
    | Prazno -> 0
    | Sestavljeno (l, _, d) -> 1 + max (visina l) (visina d)

  let razlika = function
    | Prazno -> 0
    | Sestavljeno (l, _, d) -> visina l - visina d

  let uravnotezi drevo =
    match drevo with
    | Sestavljeno (l, x, d) when razlika drevo = 2 && razlika l = 1 ->
        zavrti_desno drevo
    | Sestavljeno (l, x, d) when razlika drevo = 2 ->
        Sestavljeno (zavrti_levo l, x, d) |> zavrti_desno
    | Sestavljeno (l, x, d) when razlika drevo = -2 && razlika d = -1 ->
        zavrti_levo drevo
    | Sestavljeno (l, x, d) when razlika drevo = -2 ->
        Sestavljeno (l, x, zavrti_desno d) |> zavrti_levo
    | _ -> drevo

  let rec isci x drevo =
    match drevo with
    | Prazno -> false
    | Sestavljeno (l, vrednost, d) ->
        if x < vrednost then isci x l
        else if x > vrednost then isci x d
        else true

  let rec dodaj x drevo =
    match drevo with
    | Prazno -> Sestavljeno (Prazno, x, Prazno)
    | Sestavljeno (l, vrednost, d) ->
        if x < vrednost then Sestavljeno (dodaj x l, vrednost, d) |> uravnotezi
        else if x > vrednost then
          Sestavljeno (l, vrednost, dodaj x d) |> uravnotezi
        else drevo
end

module type AVL =
  sig
    type 'a t
    val prazna : 'a t
    val vsebuje : 'a t -> 'a -> bool
    val velikost : 'a t -> int
    val podvoji : 'a t -> 'a t
    val zavrti_levo : 'a t -> 'a t
    val zavrti_desno : 'a t -> 'a t
    val visina : 'a t -> int
    val razlika : 'a t -> int
    val uravnotezi : 'a t -> 'a t
    val isci : 'a -> 'a t -> bool
    val dodaj : 'a -> 'a t -> 'a t
  end


module Avl : AVL


In [26]:
module type MACHINE = sig
  type t
  val make : state -> state list -> t
  val initial : t -> state
  val add_transition : state -> char -> state -> char -> direction -> t -> t
  val step : t -> state -> Tape.t -> (state * Tape.t) option
end
module Machine : MACHINE = struct
  type t = { simboli : char list;
             prazen : char;
             stanja : state list;
             zacetno : state;
            gama : ((char * state) * (char * state * direction)) Avl.t
           }
   let make st states =
     {
       simboli = ['0';'1'];
       prazen = ' ';
       stanja = states;
       zacetno = st;
       gama = Avl.prazna
      }
  let initial t = t.zacetno
  let add_transition current_st read_ch next_st written_ch dir t =
    let new_transition = ((read_ch, current_st), (written_ch, next_st, dir)) in
    {t with gama = Avl.dodaj new_transition t.gama}
  
  let step machine current_state tape =
    let read_char = Tape.read tape in
    let key = (read_char, current_state) in
    match Avl.isci key machine.gama with (*nekaj je se treba popravit pri tem tipu vendar sem se raje odlocil, da oddam s tro dnevno zmaudo kot stiri dnevno :') *)
    | false ->
        None
    | true ->
        let updated_tape = tape |> Tape.write new_char |> Tape.move direction in
        Some (new_state, updated_tape)
    
    
    

end

error: compile_error

Primer stroja "Binary Increment" na <http://turingmachine.io> lahko implementiramo kot:

In [7]:
let binary_increment =
  Machine.(
    make "right" [ "carry"; "done" ]
    |> add_transition "right" '1' "right" '1' Right
    |> add_transition "right" '0' "right" '0' Right
    |> add_transition "right" ' ' "carry" ' ' Left
    |> add_transition "carry" '1' "carry" '0' Left
    |> add_transition "carry" '0' "done" '1' Left
    |> add_transition "carry" ' ' "done" '1' Left
  )


val binary_increment : Machine.t = <abstr>


Zapišite funkciji `slow_run` in `speed_run` tipa `Machine.t -> str -> unit`, ki simulirata Turingov stroj na traku, na katerem je na začetku zapisan dani niz. Prva naj izpiše trakove in stanja pri vseh vmesnih korakih, druga pa naj izpiše le končni trak. Slednjo bomo uporabljali tudi pri meritvi učinkovitosti izvajanja.

In [9]:
let primer_slow_run =
  slow_run binary_increment "1011"

1011
^
right
1011
 ^
right
1011
  ^
right
1011
   ^
right
1011
    ^
right
1011
   ^
carry
1010
  ^
carry
1000
 ^
carry
1100
^
done


val primer_slow_run : unit = ()


In [11]:
let primer_speed_run =
  speed_run binary_increment "1011"

1100
^


val primer_speed_run : unit = ()


## Krajši zapis

Ko definiramo Turingov stroj, prehode običajno združujemo najprej po stanjih, nato pa še po znakih. Prav tako pri dosti prehodih samo premikamo glavo, trak in stanje pa pustimo pri miru. Zapišite funkcije:

- `for_state`
- `for_character`
- `for_characters`
- `move`
- `switch_and_move`
- `write_and_move`
- `write_switch_and_move`

s katerimi bi lahko zgornji primer na krajše zapisali kot spodaj. Implementacijo in tipe ugotovite sami.

In [13]:
let binary_increment' =
  Machine.make "right" ["carry"; "done"]
  |> for_state "right" [
    for_characters "01" @@ move Right;
    for_character ' ' @@ switch_and_move "carry" Left
  ]
  |> for_state "carry" [
    for_character '1' @@ write_and_move '0' Left;
    for_characters "0 " @@ write_switch_and_move '1' "done" Left
  ]  

val binary_increment' : Machine.t = <abstr>


## Primeri Turingovih strojev

Pri tej nalogi boste sestavljali stroje, ki bodo iz začetnega niza na traku na različne načine izračunali nov niz. Pri tem lahko predpostavite, da je začetni niz sestavljen iz ničel in enic, preostanek traku pa je prazen. Na koncu izvajanja naj bo glava na začetku novega niza, z izjemo tega niza pa naj bo trak prazen. Ni pa treba, da se izračunani niz začne na istem mestu na traku, kot se je začel prvotni niz.

### Obračanje niza


Sestavite Turingov stroj, ki začetni niz obrne na glavo.

In [15]:
let primer_reverse = speed_run reverse "0000111001"

1001110000          
^


val primer_reverse : unit = ()


### Podvajanje niza


Sestavite Turingov stroj, ki podvoji začetni niz.

In [17]:
let primer_duplicate = speed_run duplicate "010011"

001100001111       
^


val primer_duplicate : unit = ()


### Eniški zapis


Sestavite Turingov stroj, ki na začetku na traku sprejme število $n$, zapisano v dvojiškem zapisu, na koncu pa naj bo na traku zapisanih natanko $n$ enic.

In [19]:
let primer_to_unary = speed_run to_unary "1010"

1111111111
^


val primer_to_unary : unit = ()


### Dvojiški zapis


Sestavite ravno obratni Turingov stroj, torej tak, ki na začetku na traku sprejme število $n$ enic, na koncu pa naj bo na traku zapisano število $n$ v dvojiškem zapisu.

In [21]:
let primer_to_binary = speed_run to_binary (String.make 42 '1')

val primer_to_binary : unit = ()


101010                                           
^
